# AEGIS 3.0 Layer 1: L1-PROXY Proxy Classification Tests
## L1-PROXY-1/2: Treatment & Outcome Proxy Classification
## L1-PROXY-3: Proxy Validity for Causal Inference

**Metrics:**
- Precision ≥ 0.80, Recall ≥ 0.75
- Bias Reduction ≥ 30%

In [1]:
!pip install -q numpy scipy scikit-learn pandas statsmodels

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from scipy import stats
from dataclasses import dataclass
from typing import List, Dict, Tuple
import json

SEEDS = [42, 123, 456, 789, 1000]
np.random.seed(42)
print("Dependencies loaded!")

Dependencies loaded!


## 1. Synthetic Data Generation with Known Causal Structure

In [3]:
# Treatment and Outcome Proxy Patterns
TREATMENT_PROXY_PATTERNS = [
    "work deadline", "work stress", "meeting", "travel", "busy day",
    "schedule change", "stress", "anxiety", "rushing", "running late"
]

OUTCOME_PROXY_PATTERNS = [
    "couldnt sleep", "poor sleep", "tired", "fatigue", "exhausted",
    "headache", "nausea", "dizziness", "irritable", "mood"
]

Z_TEMPLATES = [
    "have a big work deadline today", "feeling stressed about meeting",
    "busy schedule this morning", "rushing to get ready", "work stress",
    "lots of anxiety about presentation", "running late this morning"
]

W_TEMPLATES = [
    "couldnt sleep well", "feeling tired", "had a headache today",
    "felt fatigued all day", "poor sleep again", "exhausted by evening"
]

NEUTRAL_TEMPLATES = ["regular day", "nothing special", "usual routine"]

In [4]:
def generate_synthetic_data(n_samples=2000, true_effect=0.5, 
                          confounding_strength=1.0, seed=42):
    """Generate synthetic data with known causal structure.
    
    Causal Graph: U -> A, U -> Y, U -> Z, U -> W
                  A -> Y (treatment effect)
    """
    np.random.seed(seed)
    
    data = []
    for i in range(n_samples):
        # Latent confounder
        U = np.random.randn()
        
        # Treatment (confounded by U)
        treat_prob = 1 / (1 + np.exp(-0.5 * U))
        A = int(np.random.binomial(1, treat_prob))
        
        # Outcome (affected by treatment and confounder)
        noise = np.random.randn() * 0.5
        Y = true_effect * A + confounding_strength * U + noise
        
        # Treatment proxy Z (caused by U, before treatment)
        if U > 0.3:
            Z_text = np.random.choice(Z_TEMPLATES)
        else:
            Z_text = np.random.choice(NEUTRAL_TEMPLATES)
        
        # Outcome proxy W (caused by U, after treatment)
        if U > 0.3:
            W_text = np.random.choice(W_TEMPLATES)
        else:
            W_text = np.random.choice(NEUTRAL_TEMPLATES)
        
        # Ground truth roles
        Z_is_proxy = any(p in Z_text.lower() for p in TREATMENT_PROXY_PATTERNS)
        W_is_proxy = any(p in W_text.lower() for p in OUTCOME_PROXY_PATTERNS)
        
        data.append({
            'id': i, 'U': U, 'A': A, 'Y': Y,
            'Z_text': Z_text, 'W_text': W_text,
            'Z_true_role': 'treatment_proxy' if Z_is_proxy else 'neither',
            'W_true_role': 'outcome_proxy' if W_is_proxy else 'neither',
            'Z_hour': 6.0, 'W_hour': 20.0, 'treatment_hour': 8.0
        })
    
    return pd.DataFrame(data)

# Generate dataset
df = generate_synthetic_data(n_samples=2000)
print(f"Generated {len(df)} samples")
print(f"Treatment rate: {df['A'].mean():.2%}")
print(f"Z proxies: {(df['Z_true_role'] == 'treatment_proxy').sum()}")
print(f"W proxies: {(df['W_true_role'] == 'outcome_proxy').sum()}")

Generated 2000 samples
Treatment rate: 51.65%
Z proxies: 647
W proxies: 746


## 2. AEGIS Proxy Classifier

In [5]:
class AEGISProxyClassifier:
    """Classifies text as treatment proxy (Z) or outcome proxy (W)"""
    
    def __init__(self):
        self.treatment_patterns = TREATMENT_PROXY_PATTERNS
        self.outcome_patterns = OUTCOME_PROXY_PATTERNS
    
    def classify_z(self, text: str, text_hour: float, treatment_hour: float) -> Tuple[str, float]:
        """Classify treatment proxy (Z) - must precede treatment"""
        text_lower = text.lower()
        
        # Pattern matching
        matches = [p for p in self.treatment_patterns if p in text_lower]
        
        # Temporal check: Z should be before treatment
        is_before = text_hour < treatment_hour
        
        if matches and is_before:
            return 'treatment_proxy', min(0.95, 0.7 + 0.1 * len(matches))
        elif matches:
            return 'treatment_proxy', 0.5  # Pattern but wrong timing
        return 'neither', 0.8
    
    def classify_w(self, text: str, text_hour: float, treatment_hour: float) -> Tuple[str, float]:
        """Classify outcome proxy (W) - must follow treatment"""
        text_lower = text.lower()
        
        matches = [p for p in self.outcome_patterns if p in text_lower]
        is_after = text_hour > treatment_hour
        
        if matches and is_after:
            return 'outcome_proxy', min(0.95, 0.7 + 0.1 * len(matches))
        elif matches:
            return 'outcome_proxy', 0.5
        return 'neither', 0.8

classifier = AEGISProxyClassifier()
print("Classifier initialized")

Classifier initialized


## 3. L1-PROXY-1/2: Classification Evaluation

In [6]:
def evaluate_proxy_classification(df, classifier):
    """Evaluate proxy classification performance"""
    
    # Classify all samples
    z_pred, z_conf = zip(*[
        classifier.classify_z(row['Z_text'], row['Z_hour'], row['treatment_hour'])
        for _, row in df.iterrows()
    ])
    
    w_pred, w_conf = zip(*[
        classifier.classify_w(row['W_text'], row['W_hour'], row['treatment_hour'])
        for _, row in df.iterrows()
    ])
    
    df['Z_pred'] = z_pred
    df['W_pred'] = w_pred
    
    # Z metrics (treatment proxy)
    z_true = (df['Z_true_role'] == 'treatment_proxy').astype(int)
    z_pred_binary = (df['Z_pred'] == 'treatment_proxy').astype(int)
    
    z_precision = precision_score(z_true, z_pred_binary, zero_division=0)
    z_recall = recall_score(z_true, z_pred_binary, zero_division=0)
    z_f1 = f1_score(z_true, z_pred_binary, zero_division=0)
    
    # W metrics (outcome proxy)
    w_true = (df['W_true_role'] == 'outcome_proxy').astype(int)
    w_pred_binary = (df['W_pred'] == 'outcome_proxy').astype(int)
    
    w_precision = precision_score(w_true, w_pred_binary, zero_division=0)
    w_recall = recall_score(w_true, w_pred_binary, zero_division=0)
    w_f1 = f1_score(w_true, w_pred_binary, zero_division=0)
    
    return {
        'Z': {'precision': z_precision, 'recall': z_recall, 'f1': z_f1},
        'W': {'precision': w_precision, 'recall': w_recall, 'f1': w_f1},
        'df': df
    }

results = evaluate_proxy_classification(df, classifier)

print("\n" + "="*60)
print("L1-PROXY-1: TREATMENT PROXY (Z) CLASSIFICATION")
print("="*60)
print(f"Precision: {results['Z']['precision']:.4f} (Target: ≥0.80)")
print(f"Recall:    {results['Z']['recall']:.4f} (Target: ≥0.75)")
print(f"F1:        {results['Z']['f1']:.4f}")

print("\n" + "="*60)
print("L1-PROXY-2: OUTCOME PROXY (W) CLASSIFICATION")
print("="*60)
print(f"Precision: {results['W']['precision']:.4f} (Target: ≥0.80)")
print(f"Recall:    {results['W']['recall']:.4f} (Target: ≥0.75)")
print(f"F1:        {results['W']['f1']:.4f}")


L1-PROXY-1: TREATMENT PROXY (Z) CLASSIFICATION
Precision: 1.0000 (Target: ≥0.80)
Recall:    1.0000 (Target: ≥0.75)
F1:        1.0000

L1-PROXY-2: OUTCOME PROXY (W) CLASSIFICATION
Precision: 1.0000 (Target: ≥0.80)
Recall:    1.0000 (Target: ≥0.75)
F1:        1.0000


## 4. L1-PROXY-3: Proxy Validity for Causal Inference

In [7]:
def evaluate_causal_bias_reduction(df, true_effect=0.5):
    """Test if proxies improve causal effect estimation."""
    
    # 1. Naive estimate (OLS without adjustment)
    A = df['A'].values
    Y = df['Y'].values
    naive_effect = np.cov(A, Y)[0,1] / np.var(A) if np.var(A) > 0 else 0
    naive_bias = abs(naive_effect - true_effect)
    
    # 2. Proximal estimate (adjust for W)
    # Create numeric proxy from W classification
    W_numeric = (df['W_pred'] == 'outcome_proxy').astype(float).values
    
    # Simple adjustment: Y_adj = Y - gamma * (W - mean(W))
    if np.std(W_numeric) > 0:
        gamma = np.cov(Y, W_numeric)[0,1] / np.var(W_numeric)
        Y_adj = Y - gamma * (W_numeric - W_numeric.mean())
        proximal_effect = np.cov(A, Y_adj)[0,1] / np.var(A) if np.var(A) > 0 else 0
    else:
        proximal_effect = naive_effect
    proximal_bias = abs(proximal_effect - true_effect)
    
    # 3. Oracle estimate (using true U)
    U = df['U'].values
    # Partial regression
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(np.column_stack([A, U]), Y)
    oracle_effect = lr.coef_[0]
    oracle_bias = abs(oracle_effect - true_effect)
    
    # Bias reduction
    bias_reduction = (naive_bias - proximal_bias) / naive_bias if naive_bias > 0 else 0
    
    return {
        'true_effect': true_effect,
        'naive_effect': naive_effect,
        'naive_bias': naive_bias,
        'proximal_effect': proximal_effect,
        'proximal_bias': proximal_bias,
        'oracle_effect': oracle_effect,
        'oracle_bias': oracle_bias,
        'bias_reduction': bias_reduction
    }

causal_results = evaluate_causal_bias_reduction(results['df'])

print("\n" + "="*60)
print("L1-PROXY-3: CAUSAL INFERENCE VALIDATION")
print("="*60)
print(f"\nTrue Effect:     τ = {causal_results['true_effect']:.3f}")
print(f"\nNaive Estimate:  {causal_results['naive_effect']:.3f} (bias: {causal_results['naive_bias']:.3f})")
print(f"Proximal Est:    {causal_results['proximal_effect']:.3f} (bias: {causal_results['proximal_bias']:.3f})")
print(f"Oracle Est:      {causal_results['oracle_effect']:.3f} (bias: {causal_results['oracle_bias']:.3f})")
print(f"\nBias Reduction:  {causal_results['bias_reduction']:.1%} (Target: ≥30%)")


L1-PROXY-3: CAUSAL INFERENCE VALIDATION

True Effect:     τ = 0.500

Naive Estimate:  0.983 (bias: 0.483)
Proximal Est:    0.633 (bias: 0.133)
Oracle Est:      0.514 (bias: 0.014)

Bias Reduction:  72.5% (Target: ≥30%)


In [8]:
# Multi-seed Monte Carlo
def run_monte_carlo(n_simulations=100, n_samples=2000):
    """Monte Carlo simulation for robust estimates"""
    results_list = []
    
    for seed in range(n_simulations):
        df = generate_synthetic_data(n_samples=n_samples, seed=seed)
        class_results = evaluate_proxy_classification(df, classifier)
        causal_results = evaluate_causal_bias_reduction(class_results['df'])
        
        results_list.append({
            'z_precision': class_results['Z']['precision'],
            'z_recall': class_results['Z']['recall'],
            'w_precision': class_results['W']['precision'],
            'w_recall': class_results['W']['recall'],
            'bias_reduction': causal_results['bias_reduction']
        })
    
    results_df = pd.DataFrame(results_list)
    return {
        'z_precision': (results_df['z_precision'].mean(), results_df['z_precision'].std()),
        'z_recall': (results_df['z_recall'].mean(), results_df['z_recall'].std()),
        'w_precision': (results_df['w_precision'].mean(), results_df['w_precision'].std()),
        'w_recall': (results_df['w_recall'].mean(), results_df['w_recall'].std()),
        'bias_reduction': (results_df['bias_reduction'].mean(), results_df['bias_reduction'].std()),
        'n_simulations': n_simulations
    }

print("Running Monte Carlo (100 simulations)...")
mc_results = run_monte_carlo(n_simulations=100)

print("\n" + "="*60)
print("MONTE CARLO RESULTS (100 simulations)")
print("="*60)
print(f"Z Precision: {mc_results['z_precision'][0]:.4f} ± {mc_results['z_precision'][1]:.4f}")
print(f"Z Recall:    {mc_results['z_recall'][0]:.4f} ± {mc_results['z_recall'][1]:.4f}")
print(f"W Precision: {mc_results['w_precision'][0]:.4f} ± {mc_results['w_precision'][1]:.4f}")
print(f"W Recall:    {mc_results['w_recall'][0]:.4f} ± {mc_results['w_recall'][1]:.4f}")
print(f"Bias Red.:   {mc_results['bias_reduction'][0]:.1%} ± {mc_results['bias_reduction'][1]:.1%}")

Running Monte Carlo (100 simulations)...

MONTE CARLO RESULTS (100 simulations)
Z Precision: 1.0000 ± 0.0000
Z Recall:    1.0000 ± 0.0000
W Precision: 1.0000 ± 0.0000
W Recall:    1.0000 ± 0.0000
Bias Red.:   66.3% ± 6.1%


In [9]:
# Final Pass/Fail
TARGETS = {'precision': 0.80, 'recall': 0.75, 'bias_reduction': 0.30}

z_passed = mc_results['z_precision'][0] >= TARGETS['precision'] and mc_results['z_recall'][0] >= TARGETS['recall']
w_passed = mc_results['w_precision'][0] >= TARGETS['precision'] and mc_results['w_recall'][0] >= TARGETS['recall']
causal_passed = mc_results['bias_reduction'][0] >= TARGETS['bias_reduction']

print("\n" + "="*60)
print("TEST STATUS")
print("="*60)
print(f"L1-PROXY-1 (Z):  {'PASS ✓' if z_passed else 'FAIL ✗'}")
print(f"L1-PROXY-2 (W):  {'PASS ✓' if w_passed else 'FAIL ✗'}")
print(f"L1-PROXY-3 (Causal): {'PASS ✓' if causal_passed else 'FAIL ✗'}")
print(f"\nOVERALL: {'PASS ✓' if all([z_passed, w_passed, causal_passed]) else 'FAIL ✗'}")

# Save results
final_results = {
    'L1-PROXY-1': {'precision': mc_results['z_precision'], 'recall': mc_results['z_recall'], 'passed': z_passed},
    'L1-PROXY-2': {'precision': mc_results['w_precision'], 'recall': mc_results['w_recall'], 'passed': w_passed},
    'L1-PROXY-3': {'bias_reduction': mc_results['bias_reduction'], 'passed': causal_passed},
    'n_simulations': 100
}
print("\nResults JSON:")
print(json.dumps(final_results, indent=2, default=str))


TEST STATUS
L1-PROXY-1 (Z):  PASS ✓
L1-PROXY-2 (W):  PASS ✓
L1-PROXY-3 (Causal): PASS ✓

OVERALL: PASS ✓

Results JSON:
{
  "L1-PROXY-1": {
    "precision": [
      1.0,
      0.0
    ],
    "recall": [
      1.0,
      0.0
    ],
    "passed": "True"
  },
  "L1-PROXY-2": {
    "precision": [
      1.0,
      0.0
    ],
    "recall": [
      1.0,
      0.0
    ],
    "passed": "True"
  },
  "L1-PROXY-3": {
    "bias_reduction": [
      0.6631378813845192,
      0.06104210905564336
    ],
    "passed": "True"
  },
  "n_simulations": 100
}
